In [1]:
from utils import ftextraction
import pandas as pd
import os
%config IPCompleter.greedy=True
import gc
gc.enable()

In [5]:
os.listdir('data')

['application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'credit_card_balance.csv',
 'HomeCredit_columns_description.csv',
 'installments_payments.csv',
 'POS_CASH_balance.csv',
 'previous_application.csv',
 'sample_submission.csv']

In [31]:
extractor = ftextraction.Extractor()
def df_info(df):
    print(f'Data frame shape: {df.shape}')
    print('Memory usage: %.2f MB' %(df.memory_usage().sum()/(1024*1024)))
    
def missing_values(df):
    missing = df.isnull().sum().sort_values(ascending=False)
    count = missing[missing != 0]
    prcnt = (count/len(df)*100).round(1)
    return pd.concat([count,prcnt],axis=1,keys=['Count','Percent'])

### Bureau 

In [27]:
buro = pd.read_csv('data/bureau.csv')
buro_num = extractor.numerical_feature_stats(buro,'SK_ID_CURR','buro',exclude=['SK_ID_BUREAU'])
buro_num.to_csv('processed/bureau_num.csv')
buro_cat = extractor.categorical_stats(buro,'SK_ID_CURR','buro',exclude=['SK_ID_BUREAU'])
buro_cat.to_csv('processed/bureau_cat.csv')

### Bureau Balance Data 

In [108]:
buro_b = pd.read_csv('data/bureau_balance.csv')
bb_num = extractor.numerical_feature_stats(buro_b,'SK_ID_BUREAU','bb')
bb_cat = extractor.categorical_stats(buro_b,'SK_ID_BUREAU','bb')
bb = pd.merge(bb_num,bb_cat,on='SK_ID_BUREAU',how='outer')
bb_client = pd.merge(buro[['SK_ID_CURR','SK_ID_BUREAU']],bb,on='SK_ID_BUREAU',how='left')
bb_client = extractor.numerical_feature_stats(bb_client,'SK_ID_CURR','client',exclude=['SK_ID_BUREAU'])
bb_client.to_csv('processed/bureau_balance_by_client.csv')

### Previous Applications Data 

In [127]:
p_apps = pd.read_csv('data/previous_application.csv')
p_apps_numerical = extractor.numerical_feature_stats(p_apps,'SK_ID_CURR','prv_app',exclude=['SK_ID_PREV'])
p_app_cat = extractor.categorical_stats(p_apps,'SK_ID_CURR','prv_app',exclude=['SK_ID_PREV'])
p_app_cat.to_csv('processed/p_apps_cat.csv')
p_apps_numerical.to_csv('processed/p_apps_num.csv')

### Installments Payments

In [20]:
ins_pay = pd.read_csv('data/installments_payments.csv')
ins_pay_loan = extractor.numerical_feature_stats(ins_pay,'SK_ID_PREV','ins_pay',exclude=['SK_ID_CURR'])
ins_pay_loan_client = pd.merge(p_apps[['SK_ID_PREV','SK_ID_CURR']],ins_pay_loan,how='left',on='SK_ID_PREV')

In [34]:
#INSTALLMENTS DATA BY CLIENT
ins_pay_loan_client = extractor.\
numerical_feature_stats(ins_pay_loan_client,'SK_ID_CURR','client',exclude=['SK_ID_PREV'])
ins_pay_loan_client.to_csv('ins_pay_loan_client.csv')

### Credit Card Balance data: 

In [36]:
cc_balance = pd.read_csv('data/credit_card_balance.csv')
cc_balance_num = extractor.numerical_feature_stats(cc_balance,'SK_ID_PREV','ccb',exclude=['SK_ID_CURR'])
cc_balance_cat = extractor.categorical_stats(cc_balance,'SK_ID_PREV','ccb',exclude=['SK_ID_CURR'])
cc_balance_pa = pd.merge(cc_balance_num,cc_balance_cat,on='SK_ID_PREV',how='outer')
#CREDIT CARD BALANCE DATA BY CLIENT:
cc_b_client = pd.merge(p_apps[['SK_ID_PREV','SK_ID_CURR']],cc_balance_pa,on='SK_ID_PREV',how='left')
cc_b_client = extractor.numerical_feature_stats(cc_b_client,'SK_ID_CURR','client',exclude=['SK_ID_PREV'])
cc_b_client.to_csv('cc_b_client.csv')

### POS Cash Balance Data 

In [61]:
pos_cb = pd.read_csv('data/POS_CASH_balance.csv')
pos_cb_num = extractor.numerical_feature_stats(pos_cb,'SK_ID_PREV','poscb',exclude=['SK_ID_CURR'])
pos_cb_cat = extractor.categorical_stats(pos_cb,'SK_ID_PREV','poscb',exclude=['SK_ID_CURR'])
pos_cb_loan = pd.merge(pos_cb_num,pos_cb_cat,on='SK_ID_PREV',how='outer')
pos_cb_client = pd.merge(p_apps[['SK_ID_PREV','SK_ID_CURR']],pos_cb_loan,on='SK_ID_PREV',how='left')
pos_cb_client = extractor.numerical_feature_stats(pos_cb_client,'SK_ID_CURR','client',exclude=['SK_ID_PREV'])
pos_cb_client.to_csv('pos_cb_client.csv')

### Joining Databases:

In [3]:
ins_pay_loan_client = pd.read_csv('processed/ins_pay_loan_client.csv')
cc_b_client = pd.read_csv('processed/cc_b_client.csv')
pos_cb_client = pd.read_csv('processed/pos_cb_client.csv')
p_apps_num = pd.read_csv('processed/p_apps_num.csv')
p_apps_cat = pd.read_csv('processed/p_apps_cat.csv')
bureau_balance_by_client = pd.read_csv('processed/bureau_balance_by_client.csv')
bureau_num = pd.read_csv('processed/bureau_num.csv')
bureau_cat = pd.read_csv('processed/bureau_cat.csv')

In [6]:
print('Installments Database...\n')
df_info(ins_pay_loan_client)
print('Credit Card Balance Database...\n')
df_info(cc_b_client)
print('POS Cash Balance...\n')
df_info(pos_cb_client)
print('Previous Applications Numerical...\n')
df_info(p_apps_num)
print('Previous Applications Categorical...\n')
df_info(p_apps_cat)
print('Bureau Balance Database...\n')
df_info(bureau_balance_by_client)
print('Bureau Numerical.....\n')
df_info(bureau_num)
print('Bureau Categorical....\n')
df_info(bureau_cat)

Installments Database...

Data frame shape: (338857, 152)
Memory usage: 392.96 MB
Credit Card Balance Database...

Data frame shape: (338857, 572)
Memory usage: 1478.78 MB
POS Cash Balance...

Data frame shape: (338857, 217)
Memory usage: 561.00 MB
Previous Applications Numerical...

Data frame shape: (338857, 97)
Memory usage: 250.77 MB
Previous Applications Categorical...

Data frame shape: (338857, 288)
Memory usage: 744.56 MB
Bureau Balance Database...

Data frame shape: (305811, 107)
Memory usage: 249.65 MB
Bureau Numerical.....

Data frame shape: (305811, 62)
Memory usage: 144.66 MB
Bureau Categorical....

Data frame shape: (305811, 48)
Memory usage: 111.99 MB


##### DROPPING COLUMNS WITH HIGH MISSING VALUES:

In [7]:
def drop_high_missing(df):
    missing = missing_values(df)
    cols = list(missing.loc[missing['Percent']>80].index)
    df = df.drop(columns=cols)
    print('Dropped %d columns'%len(cols))

In [10]:
all_data = [ins_pay_loan_client,cc_b_client,pos_cb_client,p_apps_num,p_apps_cat,bureau_balance_by_client,bureau_num,bureau_cat]

In [11]:
#DROPPING COLUMNS
for dataset in all_data:
    drop_high_missing(dataset)

Dropped 0 columns
Dropped 63 columns
Dropped 0 columns
Dropped 6 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns


In [12]:
# LOADING MAIN TRAINING AND TESTING DATASETS:
app_train = pd.read_csv('data/application_train.csv')
app_test = pd.read_csv('data/application_test.csv')

In [13]:
# JOINING INSTALLMENTS PAYMENTS DATA:
app_train = pd.merge(app_train,ins_pay_loan_client,on='SK_ID_CURR',how='left')
#JOINING CREDIT CARD BALANCE DATA:
app_train = pd.merge(app_train,cc_b_client,on='SK_ID_CURR',how='left')
#JOINING POS BALANCE DATA:
app_train = pd.merge(app_train,pos_cb_client,on='SK_ID_CURR',how='left')
#JOINING PREVIOUS APPLICATIONS CATEGORICAL
app_train = pd.merge(app_train,p_apps_cat,on='SK_ID_CURR',how='left')
#JOINING PREVIOUS APPLICATIONS NUMERICAL
app_train = pd.merge(app_train,p_apps_num,on='SK_ID_CURR',how='left')
#JOINING BUREAU NUMERICAL DATA:
app_train = pd.merge(app_train,bureau_num,on='SK_ID_CURR',how='left')
#JOINING BUREAU CATEGORICAL DATA:
app_train = pd.merge(app_train,bureau_cat,on='SK_ID_CURR',how='left')
#JOINING BUREAU BALANCE DATA:
app_train = pd.merge(app_train,bureau_balance_by_client,on='SK_ID_CURR',how='left')

#SAVING DATABASE:
app_train.to_csv('app_train_all.csv')

In [14]:
# JOINING INSTALLMENTS PAYMENTS DATA:
app_test = pd.merge(app_test,ins_pay_loan_client,on='SK_ID_CURR',how='left')
#JOINING CREDIT CARD BALANCE DATA:
app_test = pd.merge(app_test,cc_b_client,on='SK_ID_CURR',how='left')
#JOINING POS BALANCE DATA:
app_test = pd.merge(app_test,pos_cb_client,on='SK_ID_CURR',how='left')
#JOINING PREVIOUS APPLICATIONS CATEGORICAL
app_test = pd.merge(app_test,p_apps_cat,on='SK_ID_CURR',how='left')
#JOINING PREVIOUS APPLICATIONS NUMERICAL
app_test = pd.merge(app_test,p_apps_num,on='SK_ID_CURR',how='left')
#JOINING BUREAU NUMERICAL DATA:
app_test = pd.merge(app_test,bureau_num,on='SK_ID_CURR',how='left')
#JOINING BUREAU CATEGORICAL DATA:
app_test = pd.merge(app_test,bureau_cat,on='SK_ID_CURR',how='left')
#JOINING BUREAU BALANCE DATA:
app_test = pd.merge(app_test,bureau_balance_by_client,on='SK_ID_CURR',how='left')
#SAVING DATABASE
app_test = app_test.to_csv('app_test_all.csv')

### Aligning training and testing data 

In [22]:
app_test = pd.read_csv('app_test_all.csv')

In [25]:
labels = app_train['TARGET']

train, test = app_train.align(app_test,join='inner',axis=1)

train['TARGET'] = labels

In [26]:
print('Training shape: ',train.shape)
print('Testing shape: ',test.shape)


Training shape:  (307511, 1657)
Testing shape:  (48744, 1656)


In [27]:
#DROP COLUMNS
train = train.drop(columns=['SK_ID_CURR','TARGET'])
test = test.drop(columns=['SK_ID_CURR'])
#ONE HOT ENCODING DATASETS:
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print('Training shape: ', train.shape)
print('Testing shape: ', test.shape)

Training shape:  (307511, 1779)
Testing shape:  (48744, 1776)


### TRAINING LGBM

In [28]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train,labels,test_size=0.25, shuffle=True,random_state=42)

print('Training set size:{}'.format(x_train.shape))
print('Validation set size:{}'.format(x_valid.shape))

Training set size:(230633, 1779)
Validation set size:(76878, 1779)


In [29]:
import lightgbm as lgb
#CREATING LGBM DATASETS:
train_set = lgb.Dataset(x_train,label=y_train)
valid_set = lgb.Dataset(x_valid,label=y_valid)

In [30]:
#hyperparameters:

params = {'boosting_type': 'gbdt', 'max_depth' : 10,
          'objective': 'binary','nthread': 5,'num_leaves': 64,
          'learning_rate': 0.05,'max_bin': 512,'subsample_for_bin': 200,
          'subsample': 1,'subsample_freq': 1,'colsample_bytree': 0.8,
          'reg_alpha': 5,'reg_lambda': 10,'min_split_gain': 0.5,
          'min_child_weight': 1,'min_child_samples': 5,
          'scale_pos_weight': 1,'num_class' : 1,
          'metric' : 'auc'
          }

In [31]:
lgbm = lgb.train(params,
                 train_set,
                 2500,
                 valid_sets=valid_set,
                 early_stopping_rounds= 40,
                 verbose_eval= 10
                 )

Training until validation scores don't improve for 40 rounds.
[10]	valid_0's auc: 0.743034
[20]	valid_0's auc: 0.751258
[30]	valid_0's auc: 0.755675
[40]	valid_0's auc: 0.759519
[50]	valid_0's auc: 0.763693
[60]	valid_0's auc: 0.767265
[70]	valid_0's auc: 0.770573
[80]	valid_0's auc: 0.7729
[90]	valid_0's auc: 0.774804
[100]	valid_0's auc: 0.776323
[110]	valid_0's auc: 0.777371
[120]	valid_0's auc: 0.778304
[130]	valid_0's auc: 0.779288
[140]	valid_0's auc: 0.779977
[150]	valid_0's auc: 0.780646
[160]	valid_0's auc: 0.781373
[170]	valid_0's auc: 0.781869
[180]	valid_0's auc: 0.78231
[190]	valid_0's auc: 0.782761
[200]	valid_0's auc: 0.78302
[210]	valid_0's auc: 0.783347
[220]	valid_0's auc: 0.783606
[230]	valid_0's auc: 0.783812
[240]	valid_0's auc: 0.784018
[250]	valid_0's auc: 0.784195
[260]	valid_0's auc: 0.784502
[270]	valid_0's auc: 0.784653
[280]	valid_0's auc: 0.784831
[290]	valid_0's auc: 0.784874
[300]	valid_0's auc: 0.784965
[310]	valid_0's auc: 0.785188
[320]	valid_0's auc: 

In [32]:
predictions = lgbm.predict(test)

In [33]:
submission = pd.read_csv('data/sample_submission.csv')
submission.TARGET = predictions
submission.to_csv('all_submission.csv',index=False)

Submission scores **0.771** on the PLB